In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
data = pd.read_csv("drug200.csv")
data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [4]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [10]:
# sex, BP, Cholesterol, Drug columns encoding

le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])
X[:, 2] = le.fit_transform(X[:, 2])
X[:, 3] = le.fit_transform(X[:, 3])
y = le.fit_transform(y)

array([0, 3, 3, 4, 0, 4, 0, 3, 0, 0, 3, 0, 0, 0, 4, 0, 4, 1, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 0, 0, 4, 2, 4, 0, 4, 4, 1, 4, 4, 4, 0, 2, 0, 4,
       4, 4, 1, 3, 0, 0, 0, 4, 0, 0, 2, 3, 2, 0, 4, 0, 0, 1, 0, 4, 2, 0,
       1, 4, 0, 0, 2, 0, 4, 0, 0, 0, 1, 0, 1, 4, 2, 4, 3, 1, 3, 2, 4, 0,
       0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 1, 1, 3, 4, 0, 4, 4, 0, 2, 0,
       1, 4, 4, 4, 4, 0, 4, 4, 1, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 4, 0, 0,
       4, 0, 0, 4, 2, 1, 2, 4, 1, 0, 2, 0, 1, 4, 4, 1, 4, 3, 1, 2, 4, 4,
       0, 3, 1, 0, 3, 4, 4, 2, 4, 0, 0, 0, 0, 4, 0, 1, 4, 4, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 4, 4, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 3, 0, 3, 3, 4,
       4, 4])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
ann = tf.keras.models.Sequential()

2022-09-20 15:55:22.333671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-20 15:55:22.333755: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-20 15:55:22.333824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (NB-LXZ15563): /proc/driver/nvidia/version does not exist
2022-09-20 15:55:22.334933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# input and hidden layers

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [18]:
# second hidden layers

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [19]:
# output layer

ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))

In [20]:
# compile

ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accura

Epoch 81/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 82/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 83/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 84/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 85/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 86/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 87/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 88/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 89/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.1063
Epoch 90/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0000e+00

In [25]:
y_pred = ann.predict(X_test)
accuracy_score(y_test, y_pred)

2/2 [==============================] - 0s 2ms/step


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets